# Создание и заполнение данных БД Postgre

In [5]:
%pip install python-dotenv psycopg2-binary


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor
from dotenv import load_dotenv


# Получение секретов

In [19]:
# получаем текущую директорию ноутбука 
current_dir = os.getcwd()

# переходим на один уровень вверх
project_root = os.path.dirname(current_dir)

# формируем путь к файлу .env в папке Task1, там у нас лежит файл .env с настройками подключения к БД
dotenv_path = os.path.join(project_root, 'task_4_DBeaver_Jupiter', '.env')

# загружаем переменные окружения из указанного файла
load_dotenv(dotenv_path)

# получим доступ к переменным окружения
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")
db_port = os.getenv("DB_PORT") 
secret_hash = os.getenv("SECRET_HASH") 

print(f"Загруженные данные: USER={user}, DB={db_name}, DB_PORT={db_port}")

Загруженные данные: USER=idrisov, DB=my_db_idrisov, DB_PORT=5433


# Подключение к базе данных PostgreSQL

In [20]:
conn = None
try:
    conn = psycopg2.connect(
        host="localhost", # если Docker контейнер запущен локально, а ноутбук вне Docker.
                          # НО! если ноутбук также в Docker и в одной сети с БД,
                          # то нужно использовать имя сервиса Docker (например, 'db' или 'postgres_db').
        database=db_name,
        user=user,
        password=password,
        port=db_port
    )
    cursor = conn.cursor()

    print("Успешное подключение к базе данных!")
    
except Exception as e:
    print(f"Ошибка при подключении к базе данных: {e}")

Успешное подключение к базе данных!


In [21]:
# пример запроса
cursor.execute("SELECT version();")
db_version = cursor.fetchone()
print(f"Версия PostgreSQL: {db_version}")

Версия PostgreSQL: ('PostgreSQL 13.23 (Debian 13.23-1.pgdg13+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit',)


In [16]:
# получить список таблиц:
cursor.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
""")
tables = cursor.fetchall()
print("\nТаблицы в базе данных:")
for table in tables:
    print(f"- {table[0]}")


Таблицы в базе данных:
- departments
- user_logs


In [17]:
# закрытие соединения с БД - После завершения работы с БД не забываем закрывать соединение!
cursor.close()
conn.close()

Вам предоставлена БД с логами (действиями) студентов на образовательном портале за весенний семестр (агрегация по каждой неделе) по отдельному электронному курсу - таблица user_logs (примечание. создана в предыдущих л.р.).
- сourseid — уникальный идентификатор курса, дисциплины;
- userid — уникальный идентификатор студента (не используется в обучении);
- num_week — номер недели в году;
- s_all — количество всех событий на текущий момент;
- s_all_avg — среднее количество всех событий в неделю;
- s_course_viewed — количество просмотров курса;
- s_course_viewed_avg — среднее количество просмотров курса в неделю;
- s_q_attempt_viewed — количество просмотров теста;
- s_q_attempt_viewed_avg — среднее количество просмотров теста в неделю;
- s_a_course_module_viewed — количество просмотров модуля в курсе;
- s_a_course_module_viewed_avg — среднее количество просмотров модуля в курсе в неделю;
- s_a_submission_status_viewed — количество отправленных заданий на проверку;
- s_a_submission_status_viewed_avg — среднее количество ответов;
- namer_level — оценка за дисциплину;
- depart — номер кафедры;
- name_osno — основа обучения (имеет два значения: бюджет или контракт);
- name_formopril — форма обучения;
- leveled — уровень образования (имеет два значения: бакалавриат, магистратура);
- num_sem — номер семестра;
- kurs — номер курса учебной группы.

Также в таблице  departments хранятся названия кафедр, таблица связана с логами по полю depart:
id - код кафедры;
name - сокращенное название кафедры. 

In [45]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor
from dotenv import load_dotenv
# получаем текущую директорию ноутбука 
current_dir = os.getcwd()

# переходим на один уровень вверх
project_root = os.path.dirname(current_dir)

# формируем путь к файлу .env в папке Task1, там у нас лежит файл .env с настройками подключения к БД
dotenv_path = os.path.join(project_root, 'task_3_DBeaver', '.env')

# загружаем переменные окружения из указанного файла
load_dotenv(dotenv_path)

# получим доступ к переменным окружения
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")
db_port = os.getenv("DB_PORT") 
secret_hash = os.getenv("SECRET_HASH") 

print(f"Загруженные данные: USER={user}, DB={db_name}, DB_PORT={db_port}")
conn = None
try:
    conn = psycopg2.connect(
        host="localhost", # если Docker контейнер запущен локально, а ноутбук вне Docker.
                          # НО! если ноутбук также в Docker и в одной сети с БД,
                          # то нужно использовать имя сервиса Docker (например, 'db' или 'postgres_db').
        database=db_name,
        user=user,
        password=password,
        port=db_port
    )
    cursor = conn.cursor()

    print("Успешное подключение к базе данных!")
    
except Exception as e:
    print(f"Ошибка при подключении к базе данных: {e}")

Загруженные данные: USER=idrisov, DB=my_db_idrisov, DB_PORT=5433
Успешное подключение к базе данных!


## Задание 1 (если до этого еще этот шаг не был выполнен):

Измените данные вещественного типа, сейчас целая и дробная часть разделены запятой, замените ее на точку. 

Выведите первые 10 записей, чтобы проверить результат предобработки. 

In [46]:
cursor.execute('''
UPDATE user_logs 
SET s_course_viewed_avg = NULLIF(REPLACE(s_course_viewed_avg::text, ',', '.'), '')::REAL,
    s_q_attempt_viewed_avg = NULLIF(REPLACE(s_q_attempt_viewed_avg::text, ',', '.'), '')::REAL,
    s_a_course_module_viewed_avg = NULLIF(REPLACE(s_a_course_module_viewed_avg::text, ',', '.'), '')::REAL,
    s_a_submission_status_viewed_avg = NULLIF(REPLACE(s_a_submission_status_viewed_avg::text, ',', '.'), '')::REAL;
    
ALTER TABLE user_logs 
    ALTER COLUMN s_course_viewed_avg TYPE REAL USING NULLIF(s_course_viewed_avg::text, '')::REAL,
    ALTER COLUMN s_q_attempt_viewed_avg TYPE REAL USING NULLIF(s_q_attempt_viewed_avg::text, '')::REAL,
    ALTER COLUMN s_a_course_module_viewed_avg TYPE REAL USING NULLIF(s_a_course_module_viewed_avg::text, '')::REAL,
    ALTER COLUMN s_a_submission_status_viewed_avg TYPE REAL USING NULLIF(s_a_submission_status_viewed_avg::text, '')::REAL;
''')
conn.commit()

In [47]:
cursor.execute('SELECT * FROM user_logs LIMIT 10;')
rows = cursor.fetchall()

print("Первые 10 записей:")
for row in rows:
    print(row)

Первые 10 записей:
(71262, 34527, 6, 9, 9.0, 4, 4.0, 0, 0.0, 0, 0.0, 0, 0.0, '3', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34609, 6, 6, 6.0, 3, 3.0, 0, 0.0, 0, 0.0, 0, 0.0, '2', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34610, 6, 13, 13.0, 5, 5.0, 0, 0.0, 1, 1.0, 1, 1.0, '5', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34611, 6, 12, 12.0, 7, 7.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экзамен', 22, '2', '1', '1', 2, 2, '18.06.2022')
(71262, 34612, 6, 24, 24.0, 8, 8.0, 0, 0.0, 0, 0.0, 0, 0.0, '3', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34613, 6, 15, 15.0, 8, 8.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34615, 6, 14, 14.0, 7, 7.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34616, 6, 5, 5.0, 2, 2.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34616, 8, 2, 3.0, 2, 2.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экз

## Задание 2: 

Выведите количество кафедр, за которыми закреплены курсы на портале.





In [48]:
cursor.execute('''
    SELECT COUNT(DISTINCT depart) 
    FROM user_logs;
''')
count = cursor.fetchone()[0] 
print(f"Количество кафедр, за которыми закреплены курсы: {count}")

Количество кафедр, за которыми закреплены курсы: 43


##  Задание 3:

Выведите сколько у каждой кафедры закреплено электронных курсов на портале. 
Требуется выводит сокращенное название кафедры и количество курсов. 
У какой кафедры больше всего курсов на портале?

In [49]:
cursor.execute(''' 
    SELECT d.name, COUNT(DISTINCT ul.courseid) as course_count
    FROM user_logs ul
    JOIN departments d ON ul.depart = d.id
    GROUP BY d.name
    ORDER BY course_count DESC;
''')
rows = cursor.fetchall()
print("Кафедры и количество курсов:")
for row in rows:
    print(f"{row[0]}: {row[1]}")

print(f"\nБольше всего курсов у: {rows[0][0]} ({rows[0][1]} курсов)")

Кафедры и количество курсов:
ДиСО: 53
ПОиД: 42
МиХТ: 42
ГМиТТК: 40
ЛиУТС: 36
БИиИТ: 35
ГМУиУП: 35
ПиЭММО: 33
АЭПиМ: 33
ЛиП: 33
ПиСЗ: 32
Эконом.: 32
ТОМ: 30
ГМДиОПИ: 29
ЛПиМ: 28
РМПИ: 28
МиТОДиМ: 28
ВТиП: 25
Психол.: 23
ВИ: 22
ТССА: 21
Дизайна: 20
Менеджм.: 20
ЯиЛ: 20
CC: 19
ПМиИ: 19
АиИИ: 19
ТиЭС: 19
Химии: 18
ХОМ: 18
РЯОЯиМК: 17
ЭПП: 16
ИиИБ: 16
АСУ: 16
Физики: 16
ЭиМЭ: 16
УиИС: 15
СРиППО: 14
ПЭиБЖД: 11
Физкульт.: 5
ЦДОМ: 4
ИТМ: 3
УСиБА: 2

Больше всего курсов у: ДиСО (53 курсов)


## Задание 4:

Ответьте на вопрос: существуют ли курсы, за которыми закреплено несколько кафедр? Если такие курсы есть, то выведите их количество.
Также выведите названия кафедр, которые совместно преподают один и тот же курс.




In [50]:
cursor.execute(''' 
    SELECT ul.courseid, array_agg(DISTINCT d.name)
    FROM user_logs ul
    JOIN departments d ON ul.depart = d.id
    GROUP BY ul.courseid
    HAVING COUNT(DISTINCT ul.depart) > 1;
''')

rows = cursor.fetchall()

if rows:
    print(f"Количество курсов, за которыми закреплено несколько кафедр: {len(rows)}\n")
    for row in rows:
        print(f"Курс {row[0]} совместно преподают кафедры: {', '.join(row[1])}")
else:
    print("Курсов, закрепленных за несколькими кафедрами, не найдено.")

Количество курсов, за которыми закреплено несколько кафедр: 60

Курс 71495 совместно преподают кафедры: ГМиТТК, ПиСЗ, УиИС
Курс 71508 совместно преподают кафедры: ПиСЗ, УиИС
Курс 71541 совместно преподают кафедры: ПиСЗ, УиИС
Курс 71547 совместно преподают кафедры: ПиСЗ, УиИС
Курс 71549 совместно преподают кафедры: ГМиТТК, ТиЭС
Курс 71571 совместно преподают кафедры: ЛиУТС, ПиСЗ, УиИС
Курс 71632 совместно преподают кафедры: CC, ВТиП
Курс 71736 совместно преподают кафедры: ГМДиОПИ, ЭиМЭ
Курс 71852 совместно преподают кафедры: АЭПиМ, ЭПП
Курс 71857 совместно преподают кафедры: АЭПиМ, ЭПП
Курс 71884 совместно преподают кафедры: АЭПиМ, ЭПП
Курс 71892 совместно преподают кафедры: АЭПиМ, ТиЭС, ЭПП
Курс 71904 совместно преподают кафедры: АЭПиМ, ЭПП
Курс 72126 совместно преподают кафедры: АЭПиМ, УиИС
Курс 72314 совместно преподают кафедры: Дизайна, ЛПиМ
Курс 72347 совместно преподают кафедры: Дизайна, ЛПиМ
Курс 72358 совместно преподают кафедры: МиХТ, ТОМ
Курс 72359 совместно преподают кафедры:

## Задание 5:

Выведите количество студентов, которые получили 2, 3, 4, 5.

Пример вывода:

| namer_level |	count |
|-----|------|
|2 |	4 |
|3 |	3435 |
|4 | 	4676765|
|5 | 232 |


In [34]:
cursor.execute(''' 
    SELECT namer_level, COUNT(DISTINCT userid)
    FROM user_logs
    WHERE namer_level IN ('2', '3', '4', '5')
    GROUP BY namer_level
    ORDER BY namer_level;
''')
rows = cursor.fetchall()

print("| namer_level | count |")
print("|---|---|")
for row in rows:
    print(f"| {row[0]} | {row[1]} |")

| namer_level | count |
|---|---|
| 2 | 1069 |
| 3 | 1884 |
| 4 | 3243 |
| 5 | 3407 |


## Задание 6:

Выведите студента, который больше всех работает на портале (у него максимальное количество логов за вест период обучения).

In [35]:
cursor.execute(''' 
    SELECT userid, MAX(s_all) as max_logs
    FROM user_logs
    GROUP BY userid
    ORDER BY max_logs DESC
    LIMIT 1;
''')
row = cursor.fetchone()
print(f"Студент с максимальной активностью: id {row[0]} (количество логов: {row[1]})")

Студент с максимальной активностью: id 21606 (количество логов: 3940)


## Задание 7:

Выведите по каждой недели среднее количество всех событий на портале.

In [36]:
cursor.execute(''' 
    SELECT num_week, AVG(s_all)
    FROM user_logs
    GROUP BY num_week
    ORDER BY num_week;
''')
rows = cursor.fetchall()

print("Неделя | Среднее кол-во событий")
for row in rows:
    print(f"{row[0]} | {row[1]:.2f}")

Неделя | Среднее кол-во событий
6 | 13.80
7 | 9.62
8 | 8.03
9 | 9.39
10 | 8.21
11 | 10.02
12 | 9.38
13 | 10.01
14 | 9.86
15 | 10.35
16 | 10.29
17 | 10.52
18 | 9.67
19 | 11.11
20 | 14.45
21 | 18.50
22 | 22.49
23 | 22.26
24 | 23.01
25 | 18.22
26 | 8.60
27 | 1.25
28 | 0.09
29 | 0.05


## Задание 8: 

Выведите название кафедры, у которой больше всего отличников.

Отдельно выведите название кафедры, у которой больше всего двоечников. 

In [51]:
# Запрос по отличникам (5)
cursor.execute(''' 
    SELECT d.name, COUNT(DISTINCT ul.userid) as cnt_5
    FROM user_logs ul
    JOIN departments d ON ul.depart = d.id
    WHERE ul.namer_level = '5'
    GROUP BY d.name
    ORDER BY cnt_5 DESC LIMIT 1;
''')
best = cursor.fetchone()

# Запрос по двоечникам (2)
cursor.execute(''' 
    SELECT d.name, COUNT(DISTINCT ul.userid) as cnt_2
    FROM user_logs ul
    JOIN departments d ON ul.depart = d.id
    WHERE ul.namer_level = '2'
    GROUP BY d.name
    ORDER BY cnt_2 DESC LIMIT 1;
''')
worst = cursor.fetchone()

print(f"Кафедра, у которой больше всего отличников: {best[0]} ({best[1]} студентов)")
print(f"Кафедра, у которой больше всего двоечников: {worst[0]} ({worst[1]} студентов)")

Кафедра, у которой больше всего отличников: ДиСО (310 студентов)
Кафедра, у которой больше всего двоечников: Эконом. (72 студентов)


## Задание 9:
Провести анализ пиковой активности студентов перед экзаменом (с использованием (Common Table Expression — CTE), оператор with).

Вывести, на какой неделе семестра студенты проявляли наибольшую активность в курсе в целом, и как эта активность распределяется между студентами-бюджетниками и контрактниками.

Пример вывода :

| name_osno | week_number	| avg_s_all	| avg_s_course_viewed |	avg_s_q_attempt_viewed |
|-----|------|------|------|------|
| бюджет |	14	| 125.45 |	45.67 |	32.12 |
|контракт |	14	| 98.76 |	38.90 |	25.43 |

In [56]:
import pandas as pd
from IPython.display import display
cursor.execute(''' 
    WITH Activity AS (
        SELECT 
               CASE name_osno::text 
                   WHEN '1' THEN 'бюджет' 
                   WHEN '2' THEN 'контракт' 
                   ELSE name_osno::text 
               END as name_osno_text, 
               num_week, 
               AVG(s_all) as avg_s_all, 
               AVG(s_course_viewed) as avg_course, 
               AVG(s_q_attempt_viewed) as avg_q
        FROM user_logs
        GROUP BY name_osno, num_week
    )
    SELECT name_osno_text, num_week, avg_s_all, avg_course, avg_q
    FROM Activity
    ORDER BY avg_s_all DESC
    LIMIT 10;
''')
rows = cursor.fetchall()
columns = ["name_osno", "week_number", "avg_s_all", "avg_s_course_viewed", "avg_s_q_attempt_viewed"]
df = pd.DataFrame(rows, columns=columns)
df = df.sort_values(by="week_number", ascending=True).reset_index(drop=True)
display(df)

,name_osno,week_number,avg_s_all,avg_s_course_viewed,avg_s_q_attempt_viewed
0,бюджет,6,16.2509067461916660,5.3450471508019666,1.00120899492222132667
1,бюджет,21,19.1902152010961554,4.0288546788103490,2.8246151366164262
2,контракт,21,16.7533401849948613,3.0454265159301131,2.1749229188078109
3,контракт,22,23.5815005138746146,4.2441932168550874,4.0719424460431655
4,бюджет,22,22.0582735552510679,4.1630531151769163,4.5078584669944386
5,контракт,23,22.7886947584789311,4.1262076053442960,3.7714285714285714
6,бюджет,23,22.0548077698073668,4.0643991295236560,4.9590553719674377
7,контракт,24,28.6877697841726619,4.7344295991778006,5.4182939362795478
8,бюджет,24,20.7865720963971951,3.7824615136616426,4.9416458450874506
9,контракт,25,23.3646454265159301,4.2464542651593011,3.8752312435765673
